## Building a Conversational Agent with Context Awareness
- Creating a conv agent that maintains context across multiple interactions.
- Many Simple chatbots lacks to maintain context, leading to disjoint and frustrating user experience.
- Solving this prob by implementing a conversational agent that can remember and refer to previous parts of the conversation.


## Components
- Language Model
- Prompt template
- History Manager
- Message Store

## Method Details

1. Setting up the environment
2. Creating the Chat History Store - to manage multiple conversation sessions.
3. Defining the Conversation Structure 
    * A system message defining the AI's role.
    * A placeholder for conversation history
    * The user's input
4. Building the Conversation Chain
5. Interacting with the agent

## Conclusion 
* Context Awareness
* Simplicity
* Flexibility
* Scalability

In [8]:
import os
from langchain_ollama import OllamaLLM
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory

* ollama pull gemma:2b

In [20]:
llm = OllamaLLM(model= 'gemma:2b',temperature= 1)

### Create a simple in-memory store for chat history

In [21]:
store = {}

def get_chat_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


### Create the prompt template

In [22]:
prompt = ChatPromptTemplate.from_messages([("system", "You are a helpful AI assistant"),
         MessagesPlaceholder(variable_name="history"),
         ("human", "{input}")
])

### Combine the prompt and model into a runnable chain

In [23]:
chain = prompt | llm

### Wrap the chain with message history

In [24]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key = "input",
    history_messages_key = "history"
)

### Example useage

In [27]:
session_id = "user_01"

response1 = chain_with_history.invoke(
    {"input": "Hello! How are you, Calculate my travel budget: 5000 * 0.8 - 750"},
    config={"configurable": {"session_id": session_id}}
)
print("AI", response1)

response2 = chain_with_history.invoke(
    {"input": "What's the result of 234 divided by 13?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI", response2)

response3 = chain_with_history.invoke(
    {"input": "What was my previous messages?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response3)

AI The AI does a great job calculating the travel budget using the provided formula! 

It clearly shows each step and the final result, making it easy to understand and follow.
AI The context does not provide any information about 234 divided by 13, so I cannot answer this question from the provided context.
AI: The context does not provide any information about previous messages, so I cannot answer this question from the provided context.


### Print the conversation history

In [29]:
print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: Hello! How are you, Calculate my travel budget: 5000 * 0.8 - 750
ai: Hello! I'm doing well, thanks for asking. I'm here to help with travel-related questions and tasks. 

What can I do for you today?
human: Hello! How are you, Calculate my travel budget: 5000 * 0.8 - 750
ai: The AI calculates the travel budget as follows:

5000 * 0.8 - 750 = 4000 - 750 = 3250.

Therefore, the travel budget is 3250.
human: Hello! How are you, Calculate my travel budget: 5000 * 0.8 - 750
ai: The AI does a great job calculating the travel budget using the provided formula! 

It clearly shows each step and the final result, making it easy to understand and follow.
human: What's the result of 234 divided by 13?
ai: The context does not provide any information about 234 divided by 13, so I cannot answer this question from the provided context.
human: What was my previous messages?
ai: The context does not provide any information about previous messages, so I cannot answer this q

In [28]:
session_id = "user_01"

# Initialize the session history if not already done
if 'session_history' not in globals():
    session_history = {}

# Function to invoke the chain with history
def invoke_chain_with_history(input_text, session_id):
    if session_id not in session_history:
        session_history[session_id] = []
    
    # Add the new input to the session history
    session_history[session_id].append(input_text)
    
    # Invoke the chain with the updated session history
    response = chain_with_history.invoke(
        {"input": input_text, "history": session_history[session_id]},
        config={"configurable": {"session_id": session_id}}
    )
    
    # Add the response to the session history
    session_history[session_id].append(response)
    
    return response

# Example usage
response1 = invoke_chain_with_history("Hello! How are you, Calculate my travel budget: 5000 * 0.8 - 750", session_id)
print("AI:", response1)

response2 = invoke_chain_with_history("What's the result of 234 divided by 13?", session_id)
print("AI:", response2)

response3 = invoke_chain_with_history("What were my previous messages?", session_id)
print("AI:", response3)


AI: Sure, here's the calculated travel budget from the context:

5000 * 0.8 - 750 = 4000 - 750 = 3250.

Therefore, the travel budget is 3250.
AI: The context does not provide any information about 234 divided by 13, so I cannot answer this question from the provided context.
AI: The context does not provide any information about previous messages, so I cannot answer these questions from the provided context.
